In [ ]:
import numpy as np
import pandas as pd
import mne
from scipy.signal import welch
from scipy.stats import skew, kurtosis

#Power Spectral Density
def band_power(signal, sfreq, fmin=0.5, fmax=45):
    freqs, psd = welch(signal, sfreq, nperseg=min(len(signal), 256))
    idx_band = np.logical_and(freqs >= fmin, freqs <= fmax)
    return np.sum(psd[idx_band])

# Spectral Entropy 
def spectral_entropy(signal, sfreq=1):
    freqs, psd = welch(signal, sfreq, nperseg=min(len(signal), 256))
    psd_norm = psd / np.sum(psd) if np.sum(psd) != 0 else np.ones_like(psd) / len(psd)
    return -np.sum(psd_norm * np.log2(psd_norm + np.finfo(float).eps))

# Variance
def variance(signal):
    return np.var(signal)

# Skewness
def signal_skewness(signal):
    return skew(signal)

# Kurtosis
def signal_kurtosis(signal):
    return kurtosis(signal)

records = []

for i in range(1, 13):
    try:
        normal_file = f"D:\\Fatigue Detection\\{i}\\{i}\\Normal state.cnt"
        fatigue_file = f"D:\\Fatigue Detection\\{i}\\{i}\\Fatigue state.cnt"

        raw_normal = mne.io.read_raw_cnt(normal_file, preload=True, verbose=False)
        raw_fatigue = mne.io.read_raw_cnt(fatigue_file, preload=True, verbose=False)

        sfreq = raw_normal.info['sfreq']

        raw_normal.notch_filter(50).filter(0.5, 45)
        raw_fatigue.notch_filter(50).filter(0.5, 45)

        # Create epochs of 1 second each
        epochs_normal = mne.make_fixed_length_epochs(raw_normal, duration=1, preload=True)
        epochs_fatigue = mne.make_fixed_length_epochs(raw_fatigue, duration=1, preload=True)

        for epoch, label in zip([epochs_normal, epochs_fatigue], ["Normal", "Fatigue"]):
            for ep in epoch.get_data():
                record = {"Subject": i, "State": label}
                variances, skewnesses, kurtoses = [], [], []
                
                for ch_idx, ch_signal in enumerate(ep):
                    psd = band_power(ch_signal, sfreq)
                    se = spectral_entropy(ch_signal, sfreq)
                    var = variance(ch_signal)
                    skewness_val = signal_skewness(ch_signal)
                    kurtosis_val = signal_kurtosis(ch_signal)
                    
                    record[f"Channel_{ch_idx+1}_PSD"] = psd
                    record[f"Channel_{ch_idx+1}_SE"] = se
                    record[f"Channel_{ch_idx+1}_Variance"] = var
                    record[f"Channel_{ch_idx+1}_Skewness"] = skewness_val
                    record[f"Channel_{ch_idx+1}_Kurtosis"] = kurtosis_val
                    
                    variances.append(var)
                    skewnesses.append(skewness_val)
                    kurtoses.append(kurtosis_val)

                # Calculate average values across all channels
                record["Avg_Variance"] = np.mean(variances)
                record["Avg_Skewness"] = np.mean(skewnesses)
                record["Avg_Kurtosis"] = np.mean(kurtoses)

                records.append(record)

        print(f"Processed Subject {i}")

    except Exception as e:
        print(f"Error processing Subject {i}: {e}")

# Convert to DataFrame and save to CSV
df = pd.DataFrame(records)
df.to_csv('eeg_features.csv', index=False)
print("Saved features")


C:\Users\Harish\AppData\Local\Temp\ipykernel_18100\2439006044.py:40: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw_normal = mne.io.read_raw_cnt(normal_file, preload=True, verbose=False)
C:\Users\Harish\AppData\Local\Temp\ipykernel_18100\2439006044.py:40: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw_normal = mne.io.read_raw_cnt(normal_file, preload=True, verbose=False)
C:\Users\Harish\AppData\Local\Temp\ipykernel_18100\2439006044.py:41: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw_fatigue = mne.io.read_raw_cnt(fatigue_file, preload=True, verbose=False)
C:\Users\Harish\AppData\Local\Temp\ipykernel_18100\2439006044.py:41: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw_fatigue = mne.io.read_raw_cnt(fatigue_file, preload=True, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.15 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.15
- Lower transition bandwidth: 0.15 Hz (-6 dB cutoff frequency: 0.07 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 22001 samples (22.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


...

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.15 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.15
- Lower transition bandwidth: 0.15 Hz (-6 dB cutoff frequency: 0.07 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 22001 samples (22.001 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Not setting metadata
600 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 600 events and 1000 original time points ...
0 bad epochs dropped
Not setting metadata
602 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 602 events and 1000 original time points ...
0 bad epochs dropped
Processed Subject 12
Saved features
